# Register LM stack to reference stack with CMTK

In [ ]:
#TODO: Adjust folder formating to fit new one


    Requirements:
	• Download MobaXterm 
	• Access permission to faimsrv01.fmi.ch or faim-gpu1.fmi.ch (Ask Enrico or Jan Eglinger)
	• Reference stack
	


	1. Structure your reference and registration files compatible to CMTK
	- Make sure you have an "images" folder with anatomical stacks converted to .nrrd format
	- And a "refbrain" folder where you have your anatomical stack also in .nrrd
	- Make sure that no previous "reformatted" or "Registration" folders are in the base path

In [ ]:
import os
import fnmatch
import shutil

import numpy as np
import SimpleITK as sitk
import skimage.io

from datetime import datetime


def get_date_from_filename(filename):
    """Extracts date from the filename and returns it as a string in the format 'YYYY-MM-DD'."""
    # Extract date string from filename
    date_str = filename.split('_')[1]

    # Convert date string to datetime object
    date = datetime.strptime(date_str, '%Y%m%d')

    # Convert datetime object back to string in the desired format
    formatted_date_str = date.strftime('%Y-%m-%d')

    return formatted_date_str


def convert_stack_tif_to_nrrd(tif_stack_path, n_channels=2, scale_factors=(1, 1, 1), output_base_path=None,
                              ref_channel=2):
    """Converts a .tif stack with n channels to .nrrd stack and scale it if needed
    tif_stack_path: str, path to tif stack
    n_channels: int, number of channels 
    scale_factors: tuple, formatted as (pixel_width, pixel_height, voxel_depth)
    output_base_path = string, target folder to store converted stack
    ref_channel: int, reference channel which should be labeled as _01.nrrd
    """
    if n_channels == 1:
        ref_channel = 1

    # Load the image
    image = skimage.io.imread(tif_stack_path)

    if output_base_path is None:
        output_base_path = os.path.splitext(tif_stack_path)[0]

    filename = os.path.basename(os.path.splitext(tif_stack_path)[0])
    print(f"Opening file {tif_stack_path}")
    # Output paths
    # output_paths = {i: f'{output_base_path}_0{i+1}.nrrd' for i in range(n_channels)}

    # Start labeling from 1
    label = 2

    for i in range(n_channels):

        # Extract channel
        channel = image[i::n_channels, :, :]

        # Convert numpy array to SimpleITK image
        sitk_image = sitk.GetImageFromArray(channel)

        # Set the image scale
        sitk_image.SetSpacing(scale_factors[::-1])

        # If current channel is the reference channel, label it as _01.nrrd
        if i + 1 == ref_channel:
            target_file_path = os.path.join(output_base_path, f'{filename}01.nrrd')
        else:
            target_file_path = os.path.join(output_base_path, f'{filename}0{label}.nrrd')
            label += 1

        # Write to .nrrd using SimpleITK
        sitk.WriteImage(sitk_image, target_file_path)
        print(f"Converted channel {i + 1} to: {target_file_path}")


# Test the function
test_this = False
if test_this:
    ref_dir = r"D:\montruth\data\20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif"
    test_dir = r"D:\montruth\data\20220414_RM0012_128hpf_fP8_f1_anatomyGFRF_001_.tif"
    test_out_dir = r"D:\montruth\napari"
    scale_factors = (0.56, 0.56, 1.0)  # pixel_width, pixel_height, voxel_depth
    convert_stack_tif_to_nrrd(ref_dir, output_base_path=test_out_dir, scale_factors=scale_factors)
    convert_stack_tif_to_nrrd(test_dir, output_base_path=test_out_dir, scale_factors=scale_factors)

# Input directory and output directory
root_dir = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData'
output_dir = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CMTK_registration\images'
# Number of channels, scale factors and reference channel
n_channels = 2
scale_factors = (0.56, 0.56, 1.0)
ref_channel = 2

# Iterate through the first-level directories (date level)
for date_folder in os.listdir(root_dir):
    date_folder_path = os.path.join(root_dir, date_folder)

    # Check if it's a directory
    if os.path.isdir(date_folder_path):
        # Iterate through the second-level directories (self level)
        for sample_folder in os.listdir(date_folder_path):
            sample_folder_path = os.path.join(date_folder_path, sample_folder)

            # Check if it's a directory
            if os.path.isdir(sample_folder_path):

                # Check if 'result/CMTK' directory exists and if it's empty
                cmtk_folder_path = os.path.join(sample_folder_path, 'results/CMTK')
                if not os.path.isdir(cmtk_folder_path) or not os.listdir(cmtk_folder_path):

                    # Check each file in the current directory
                    for filename in os.listdir(sample_folder_path):
                        # If it's a .tif file that includes "anatomyGFRF" and ends with "_"
                        if fnmatch.fnmatch(filename, '*anatomyGFRF*_.tif'):
                            # Get the path to the file
                            tif_path = os.path.join(sample_folder_path, filename)

                            convert_stack_tif_to_nrrd(tif_path, n_channels=n_channels, output_base_path=output_dir,
                                                      scale_factors=scale_factors, ref_channel=ref_channel)




	2. Connect to VM (Jan Eglinger kindly created an environment for CMTK) 
	- Open MobaXterm
	- Start local terminal 
	- Type: ssh faimsrv01.fmi.ch OR  ssh faim-gpu1.fmi.ch
	- Enter your password

	3. Run CMTK 
	- Change directory to CMTK folder:
		cd /tungstenfs/scratch/gmicro_share/_software/CMTK/cmtk-build/bin
	- Type tmux (to detach later the process from the session)
	- Set number of channels (e.g. for 2 channels -awr 0102), set parameter (here default), set reference brain path (in .nrrd file) and set registration images  (also in .nrrd)
		./munger -awr 0102 -X 52 -C 8 -G 80 -R 3 -A '--accuracy 0.4' -W '--accuracy 1.6' -s /tungstenfs/scratch/gfriedri/montruth/CMTK_registration/refbrain/20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_01.nrrd -v /tungstenfs/scratch/gfriedri/montruth/CMTK_registration/images/
	- To detach: Ctrl +b then d
	- To reattach: tmux attach
	
    - Morphed images will be stored in "reformatted" folder (most probably in a very nested structure).

In [ ]:
# Picking up morphed stacks back to original path (date>self>results>morphed_anatomy)

# Input directory and output directory
root_dir = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData'

# Assume all results are located in this directory
results_dir = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CMTK_registration\reformatted\tungstenfs\scratch\gfriedri\montruth\CMTK_registration\images'

# Iterate through the files in the results_dir
for filename in os.listdir(results_dir):
    # Extract the date and self folder from the filename
    date_str = get_date_from_filename(filename)
    
    # Get the part of the split that matches the format f<number>
    sample_folder = [part for part in filename.split('_') if part.startswith('f') and part[1:].isdigit()][0]
    
    # Get the path to the original folder
    original_folder_path = os.path.join(root_dir, date_str, sample_folder)

    # Create the 'results/CMTK' subfolder if it does not exist
    cmtk_folder_path = os.path.join(original_folder_path, 'results\morphed_anatomy')
    os.makedirs(cmtk_folder_path, exist_ok=True)

    # Move the result file from the results_dir to the 'results/CMTK' subfolder
    old_file_path = os.path.join(results_dir, filename)
    new_file_path = os.path.join(cmtk_folder_path, filename)
    shutil.move(old_file_path, new_file_path)

    print(f"Moved file {filename} to {cmtk_folder_path}")